## Pandas worked example: computing measure percentiles

In this worked example we recreate some of the analysis for the [Keppra](https://openprescribing.net/measure/keppra/) measure.

Specifically, we're going to take the practice-level prescribing data for Levetiracetam one month from BigQuery, and rank the CCGs by the ratio of Keppra vs Levetiracetam.

Along the way, we'll talk about breaking up a big problem into smaller chunks, and we'll look at [grouping](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb), [working with strings in Pandas](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.10-Working-With-Strings.ipynb), and [handling missing values](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.04-Missing-Values.ipynb).

### Setup

In [1]:
%xmode Plain

Exception reporting mode: Plain


To run this code, you will need `pandas` and `pandas-gbq` installed, and when you first run `pd.read_gbq()` you will need to authorise Pandas to access our BigQuery project.

In [2]:
import pandas as pd

### Getting the data

In [3]:
sql = '''
SELECT
  ccgs.code AS ccg,
  ccgs.name AS ccg_name,
  practices.code AS practice,
  practices.name AS practice_name,
  bnf_code,
  bnf_name,
  items
FROM
  hscic.normalised_prescribing_standard AS prescriptions
INNER JOIN
  hscic.ccgs
ON
  prescriptions.pct = ccgs.code
INNER JOIN
  hscic.practices
ON
  prescriptions.practice = practices.code
WHERE
  bnf_code LIKE '0408010A0%'
  AND month = TIMESTAMP('2018-02-01')
  AND ccgs.org_type = 'CCG'
ORDER BY
  ccg,
  practice,
  bnf_code
'''
prescriptions = pd.read_gbq(sql, project_id='ebmdatalab', dialect='standard')

### Exploring the data

In [4]:
prescriptions.head()

,ccg,ccg_name,practice,practice_name,bnf_code,bnf_name,items
0,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAAAAA,Levetiracetam_Tab 250mg,9
1,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAABAB,Levetiracetam_Tab 500mg,8
2,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAACAC,Levetiracetam_Tab 1g,8
3,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAAHAH,Levetiracetam_Oral Soln 500mg/5ml S/F,2
4,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAAIAI,Levetiracetam_Tab 750mg,6


In [5]:
prescriptions[prescriptions['practice'] == 'A83005']

,ccg,ccg_name,practice,practice_name,bnf_code,bnf_name,items
0,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAAAAA,Levetiracetam_Tab 250mg,9
1,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAABAB,Levetiracetam_Tab 500mg,8
2,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAACAC,Levetiracetam_Tab 1g,8
3,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAAHAH,Levetiracetam_Oral Soln 500mg/5ml S/F,2
4,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0AAAIAI,Levetiracetam_Tab 750mg,6
5,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0BBAAAA,Keppra_Tab 250mg,1
6,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0BBACAC,Keppra_Tab 1g,1


In [6]:
prescriptions.shape

(37412, 7)

In [7]:
prescriptions.groupby('bnf_name').size().sort_values(ascending=False)

bnf_name
Levetiracetam_Tab 500mg                   6775
Levetiracetam_Tab 250mg                   6546
Levetiracetam_Tab 1g                      5474
Levetiracetam_Tab 750mg                   4151
Levetiracetam_Oral Soln 500mg/5ml S/F     4011
Keppra_Tab 500mg                          3113
Keppra_Tab 250mg                          2245
Keppra_Tab 1g                             2008
Keppra_Oral Soln 100mg/ml                 1323
Keppra_Tab 750mg                          1037
Levetiracetam_Gran Sach 250mg S/F          251
Levetiracetam_Gran Sach 500mg S/F          237
Levetiracetam_Gran Sach 1g S/F              77
Desitrend_Gran Sach 500mg                   50
Desitrend_Gran Sach 250mg                   40
Desitrend_Gran Sach 1g                      36
Levetiracetam_I/V Inf 100mg/ml 5ml Vl       18
Levetiracetam_I/V Inf 100mg/ml 5ml Amp      14
Keppra_I/V Inf 100mg/ml 5ml Vl               3
Desitrend_Oral Soln 100mg/ml                 2
Matever_Tab 500mg                            1
dtyp

In [8]:
prescriptions.groupby('ccg')['items'].sum().sort_values(ascending=False).head()

ccg
99P    2711
11M    2410
06H    2409
11H    2248
06K    1954
Name: items, dtype: int64

### Aggregating the data

First, we'll create a new DataFrame with one row per CCG.

In [12]:
ccgs = prescriptions[['ccg', 'ccg_name']].drop_duplicates().set_index('ccg')

In [13]:
ccgs.head()

,ccg_name
ccg,
00C,NHS DARLINGTON CCG
00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG"
00J,NHS NORTH DURHAM CCG
00K,NHS HARTLEPOOL AND STOCKTON-ON-TEES CCG
00L,NHS NORTHUMBERLAND CCG


In [16]:
ccgs.loc['99P']

ccg_name    NHS NORTHERN, EASTERN AND WESTERN DEVON CCG
Name: 99P, dtype: object

Now we can add a column to our new DataFrame, which contains the total number of prescribed items for each CCG.

In [17]:
ccgs['total_items'] = prescriptions.groupby('ccg')['items'].sum()

In [18]:
ccgs.head()

,ccg_name,total_items
ccg,,
00C,NHS DARLINGTON CCG,455
00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",1410
00J,NHS NORTH DURHAM CCG,871
00K,NHS HARTLEPOOL AND STOCKTON-ON-TEES CCG,905
00L,NHS NORTHUMBERLAND CCG,957


We're interested in the Keppra prescriptions, so we want to filter the `prescriptions` DataFrame to just include rows whos `bnf_code` column starts with `"0408010A0BB`".

In [19]:
keppra_prescriptions = prescriptions[prescriptions['bnf_code'].str.startswith('0408010A0BB')]

In [20]:
keppra_prescriptions.head()

,ccg,ccg_name,practice,practice_name,bnf_code,bnf_name,items
5,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0BBAAAA,Keppra_Tab 250mg,1
6,00C,NHS DARLINGTON CCG,A83005,WHINFIELD MEDICAL PRACTICE,0408010A0BBACAC,Keppra_Tab 1g,1
14,00C,NHS DARLINGTON CCG,A83010,MOORLANDS SURGERY,0408010A0BBABAB,Keppra_Tab 500mg,1
20,00C,NHS DARLINGTON CCG,A83013,NEASHAM ROAD SURGERY,0408010A0BBAAAA,Keppra_Tab 250mg,1
21,00C,NHS DARLINGTON CCG,A83013,NEASHAM ROAD SURGERY,0408010A0BBABAB,Keppra_Tab 500mg,1


In [21]:
ccgs['keppra_items'] = keppra_prescriptions.groupby('ccg')['items'].sum()

In [22]:
ccgs.head()

,ccg_name,total_items,keppra_items
ccg,,,
00C,NHS DARLINGTON CCG,455,10
00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",1410,92
00J,NHS NORTH DURHAM CCG,871,88
00K,NHS HARTLEPOOL AND STOCKTON-ON-TEES CCG,905,45
00L,NHS NORTHUMBERLAND CCG,957,44


We can now calculate the ratio of Keppra to Levetiracetam.

In [23]:
ccgs['keppra_ratio'] = ccgs['keppra_items'] / ccgs['total_items']

In [24]:
ccgs.head()

,ccg_name,total_items,keppra_items,keppra_ratio
ccg,,,,
00C,NHS DARLINGTON CCG,455,10,0.021978
00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",1410,92,0.065248
00J,NHS NORTH DURHAM CCG,871,88,0.101033
00K,NHS HARTLEPOOL AND STOCKTON-ON-TEES CCG,905,45,0.049724
00L,NHS NORTHUMBERLAND CCG,957,44,0.045977


And we can see which CCGs perform worst...

In [25]:
ccgs.sort_values('keppra_ratio', ascending=False).head()

,ccg_name,total_items,keppra_items,keppra_ratio
ccg,,,,
05Y,NHS WALSALL CCG,882,366,0.414966
12F,NHS WIRRAL CCG,899,329,0.365962
05G,NHS NORTH STAFFORDSHIRE CCG,625,196,0.313600
00T,NHS BOLTON CCG,675,208,0.308148
05V,NHS STAFFORD AND SURROUNDS CCG,485,142,0.292784


...and best on this measure.

In [26]:
ccgs.sort_values('keppra_ratio', ascending=True).head()

,ccg_name,total_items,keppra_items,keppra_ratio
ccg,,,,
10D,NHS SWALE CCG,270,1,0.003704
00C,NHS DARLINGTON CCG,455,10,0.021978
00M,NHS SOUTH TEES CCG,1328,37,0.027861
11T,NHS NORTH SOMERSET CCG,880,27,0.030682
13T,NHS NEWCASTLE GATESHEAD CCG,1851,61,0.032955


We can calculate percentiles too.

In [27]:
ccgs['percentile'] = ccgs['keppra_ratio'].rank(pct=True)

In [28]:
ccgs.head()

,ccg_name,total_items,keppra_items,keppra_ratio,percentile
ccg,,,,,
00C,NHS DARLINGTON CCG,455,10,0.021978,0.009662
00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",1410,92,0.065248,0.212560
00J,NHS NORTH DURHAM CCG,871,88,0.101033,0.444444
00K,NHS HARTLEPOOL AND STOCKTON-ON-TEES CCG,905,45,0.049724,0.096618
00L,NHS NORTHUMBERLAND CCG,957,44,0.045977,0.077295


In [29]:
ccgs.sort_values('percentile')

,ccg_name,total_items,keppra_items,keppra_ratio,percentile
ccg,,,,,
10D,NHS SWALE CCG,270,1,0.003704,0.004831
00C,NHS DARLINGTON CCG,455,10,0.021978,0.009662
00M,NHS SOUTH TEES CCG,1328,37,0.027861,0.014493
11T,NHS NORTH SOMERSET CCG,880,27,0.030682,0.019324
13T,NHS NEWCASTLE GATESHEAD CCG,1851,61,0.032955,0.024155
06W,NHS NORWICH CCG,696,23,0.033046,0.028986
08J,NHS KINGSTON CCG,443,15,0.033860,0.033816
08Y,NHS WEST LONDON CCG,591,21,0.035533,0.038647
10C,NHS SURREY HEATH CCG,182,7,0.038462,0.043478


### Dealing with missing data

Let's look at practices in CCG 99P, and try to reproduce the above analysis.

In [30]:
prescriptions_for_99P = prescriptions[prescriptions['ccg'] == '99P']

In [31]:
prescriptions_for_99P.head()

,ccg,ccg_name,practice,practice_name,bnf_code,bnf_name,items
36654,99P,"NHS NORTHERN, EASTERN AND WESTERN DEVON CCG",L83002,HONITON SURGERY,0408010A0AAAAAA,Levetiracetam_Tab 250mg,15
36655,99P,"NHS NORTHERN, EASTERN AND WESTERN DEVON CCG",L83002,HONITON SURGERY,0408010A0AAABAB,Levetiracetam_Tab 500mg,30
36656,99P,"NHS NORTHERN, EASTERN AND WESTERN DEVON CCG",L83002,HONITON SURGERY,0408010A0AAACAC,Levetiracetam_Tab 1g,8
36657,99P,"NHS NORTHERN, EASTERN AND WESTERN DEVON CCG",L83002,HONITON SURGERY,0408010A0AAAIAI,Levetiracetam_Tab 750mg,7
36658,99P,"NHS NORTHERN, EASTERN AND WESTERN DEVON CCG",L83002,HONITON SURGERY,0408010A0BBABAB,Keppra_Tab 500mg,4


In [32]:
practices = prescriptions_for_99P[['practice', 'practice_name']].drop_duplicates().set_index('practice')

In [33]:
practices.head()

,practice_name
practice,
L83002,HONITON SURGERY
L83003,QUEEN'S MEDICAL CENTRE
L83006,ERNESETTLE PRIMARY CARE CENTRE
L83007,SEATON & COLYTON MEDICAL PRACTICE
L83008,PATHFIELDS PRACTICE


In [34]:
practices['total_items'] = prescriptions_for_99P.groupby('practice')['items'].sum()

In [35]:
practices.head()

,practice_name,total_items
practice,,
L83002,HONITON SURGERY,65
L83003,QUEEN'S MEDICAL CENTRE,16
L83006,ERNESETTLE PRIMARY CARE CENTRE,44
L83007,SEATON & COLYTON MEDICAL PRACTICE,19
L83008,PATHFIELDS PRACTICE,82


In [43]:
keppra_prescriptions_for_99P = prescriptions_for_99P[prescriptions_for_99P['bnf_code'].str.startswith('0408010A0BB')]

In [50]:
practices['keppra_items'] = keppra_prescriptions_for_99P.groupby('practice')['items'].sum()

In [51]:
practices.head()

,practice_name,total_items,keppra_items
practice,,,
L83002,HONITON SURGERY,65,4.0
L83003,QUEEN'S MEDICAL CENTRE,16,NaN
L83006,ERNESETTLE PRIMARY CARE CENTRE,44,8.0
L83007,SEATON & COLYTON MEDICAL PRACTICE,19,NaN
L83008,PATHFIELDS PRACTICE,82,5.0


What's going on here?

In [54]:
practices.fillna({'keppra_items': 0}, inplace=True)

In [55]:
practices.head()

,practice_name,total_items,keppra_items
practice,,,
L83002,HONITON SURGERY,65,4.0
L83003,QUEEN'S MEDICAL CENTRE,16,0.0
L83006,ERNESETTLE PRIMARY CARE CENTRE,44,8.0
L83007,SEATON & COLYTON MEDICAL PRACTICE,19,0.0
L83008,PATHFIELDS PRACTICE,82,5.0


In [56]:
practices['keppra_items'] = practices['keppra_items'].astype(int)

In [57]:
practices.head()

,practice_name,total_items,keppra_items
practice,,,
L83002,HONITON SURGERY,65,4
L83003,QUEEN'S MEDICAL CENTRE,16,0
L83006,ERNESETTLE PRIMARY CARE CENTRE,44,8
L83007,SEATON & COLYTON MEDICAL PRACTICE,19,0
L83008,PATHFIELDS PRACTICE,82,5


We can now continue as above.

In [58]:
df = pd.DataFrame([[1, 2, 3], [1.4, 1.5, 1.6]])

In [59]:
df

,0,1,2
0,1.0,2.0,3.0
1,1.4,1.5,1.6


In [61]:
df.transpose()

,0,1
0,1.0,1.4
1,2.0,1.5
2,3.0,1.6


In [62]:
df = df.transpose()

In [63]:
df[0]

0    1.0
1    2.0
2    3.0
Name: 0, dtype: float64

In [64]:
df[0].astype(int)

0    1
1    2
2    3
Name: 0, dtype: int64

In [65]:
df[1].astype(int)

0    1
1    1
2    1
Name: 1, dtype: int64

In [66]:
df[2] = [-1.4, -1.5, -1.6]

In [67]:
df[2]

0   -1.4
1   -1.5
2   -1.6
Name: 2, dtype: float64

In [68]:
df[2].astype(int)

0   -1
1   -1
2   -1
Name: 2, dtype: int64

In [69]:
df

,0,1,2
0,1.0,1.4,-1.4
1,2.0,1.5,-1.5
2,3.0,1.6,-1.6


In [70]:
df.unstack()

0  0    1.0
   1    2.0
   2    3.0
1  0    1.4
   1    1.5
   2    1.6
2  0   -1.4
   1   -1.5
   2   -1.6
dtype: float64

In [71]:
type(df.unstack())

pandas.core.series.Series

In [72]:
s = df.unstack()

In [73]:
s.index

MultiIndex(levels=[[0, 1, 2], [0, 1, 2]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [74]:
s[0]

0    1.0
1    2.0
2    3.0
dtype: float64

In [75]:
s[0][1]

2.0

In [76]:
s[:, 2]

0    3.0
1    1.6
2   -1.6
dtype: float64

In [77]:
s

0  0    1.0
   1    2.0
   2    3.0
1  0    1.4
   1    1.5
   2    1.6
2  0   -1.4
   1   -1.5
   2   -1.6
dtype: float64

In [78]:
s[1, 2]

1.6

In [93]:
s.loc[0:1][:, 2]

0    3.0
1    1.6
dtype: float64